<a href="https://colab.research.google.com/github/Sidd-Shanmuhavel/Job-Search-Optimization-using-NLP/blob/master/Monster_com_Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import csv
import json
import requests
import pandas as pd
import math
import os
import shutil

In [ ]:
import urllib

In [ ]:
!pip install justext
!pip install unidecode

     |████████████████████████████████| 870kB 5.4MB/s 
     |████████████████████████████████| 245kB 5.4MB/s 


In [ ]:
import cleanpage_py as cln

In [ ]:
def monsterLink(role):
  
  link = f'https://www.monster.ie/jobs/search/?q={role}'
  source = requests.get(link).text
  soup = BeautifulSoup(source,'lxml')
  num = soup.find(class_='figure')
  n = math.ceil(int(num.text.split()[0][1:])/25)
  link_final = f'https://www.monster.ie/jobs/search/?q={role}&stpage=1&page={n}'
  return link_final

In [ ]:
def getJson(link_final):
  
  source = requests.get(link_final).text
  soup = BeautifulSoup(source,'lxml')

  #fetching the search results
  job_body = soup.find(id = 'ResultsContainer')

  #It returns a json object
  script = job_body.find('script').text

  #Parsing the json, the result is a dict
  jsonValue = json.loads(script)
  return jsonValue

In [ ]:
def monsterData(jsonValue):
  
  #Links of all the jobs appearing
  link_list = []
  for job in jsonValue['itemListElement']:
    url = job['url']
    if len(url) > 0:
        link_list.append(url)
  
  #Location of all the jobs posted
  location_list = []
  role_name_list = []
  salary_list = []
  i = 1
  for link in link_list:
    # print(i)
    one_job = requests.get(link).text
    job_soup = BeautifulSoup(one_job,'lxml')
    heading = job_soup.find('div',class_='heading')

    #Location
    if heading != None:
      location_list.append(heading.find(class_ = 'location c-gray-6').text)
      #Role Name
      role_name_list.append(heading.h1.text)
    else:
      location_list.append("Location Not Found")
      role_name_list.append("Role Name not specified")
      
    #Salary
    about_job = job_soup.find('div', class_='col-md-12 col-8 ta-md-l ta-r font-semibold')
    if about_job != None:
      
      salary_list.append(about_job.text)
    else:
      salary_list.append("None")
    
  #Checking if the Salary is mentioned in the posting or nots
  for index in range(len(salary_list)):
    if any(map(str.isdigit, salary_list[index])) == False:
      salary_list[index] = 'Salary Not Quoted'
  
  #creating a dictionary of the data collected
  data = {'Role':role_name_list,'Location':location_list,'Salary':salary_list,'Job Link': link_list}
  df = pd.DataFrame(data)
  return df

In [ ]:
link = monsterLink('Data-Analyst')
jv = getJson(link)
df = monsterData(jv)

In [ ]:
df.head()

,Role,Location,Salary,Job Link
0,Data Warehouse Analyst - 50-60k,"Clare, Clare",€50000 - €60000 per year,https://job-openings.monster.ie/data-warehouse...
1,Salesforce Data Analyst,"Dublin, Dublin",€50003 - €50003 per year,https://job-openings.monster.ie/salesforce-dat...
2,SAS Data Analyst,"Dublin, Dublin",Salary Not Quoted,https://job-openings.monster.ie/sas-data-analy...
3,"Fleet, GDPR & Data Administrator/Analyst","westmeath, Westmeath",Salary Not Quoted,https://job-openings.monster.ie/fleet-gdpr-dat...
4,Data Analyst,"westmeath, Westmeath",Salary Not Quoted,https://job-openings.monster.ie/data-analyst-w...


In [ ]:
df.

In [ ]:
def download_page(link, context=None, timeout=None):
    """
    Return code, page
    0: successfully read (write to index)
    1: bad_url (write to bad_url)
    2: unicode error (write to non_ascii_urls)
    3. bad_connection_urls
    When code is not 0, return ''
    """
    try:
        req = urllib.request.Request(link)
    except ValueError as e:
        print(link, "doesn't exist.")
        return 1, ''
    except ConnectionResetError as e:
        print('ConnectionResetError', link)
        return 3, ''

    try:
        if timeout is not None:
            response = urllib.request.urlopen(
                req, context=context, timeout=timeout)
        else:
            response = urllib.request.urlopen(req, context=context)
    except UnicodeError as e:
        print('UnicodeError for', link)
        return 2, ''
    except (urllib.error.HTTPError) as e:
        print('Error {} for {}'.format(e.code, link))
        return 1, ''
    except urllib.error.URLError as e:
        print('URLError for', link)
        return 1, ''
    except http.client.HTTPException as e:
        print('HTTPException', link)
        return 1, ''
    except http.client.RemoteDisconnected as e:
        print('RemoteDisconnected', link)
        return 1, ''
    except (ConnectionError, socket.timeout) as e:
        print('ConnectionError or Timeout', link)
        return 3, ''

    try:
        page = response.read()
    except http.client.HTTPException as e:
        print('HTTPException', link)
        return 1, ''
    except (ConnectionError, socket.timeout) as e:
        print('ConnectionError or Timeout', link)
        return 3, ''
    return 0, page

In [ ]:
page = download_page(df['Job Link'][1])[1]
page

b'\n<!doctype html>\n<html>\n  <head>\n  <!-- header start -->\n  \n  <!-- req.get(\'origin-host\') = ie -->\n  <!-- header end -->\n    <meta name="google-site-verification" content="AhK0211wQ5cQW8lYNvsNLqD-yjwiOsNi-CTbWv0mUkc" />\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=yes">\n    <meta name="Description" content="Monster.com - job view">\n    \n    <title data-react-helmet="true">Monster</title>\n    <meta data-react-helmet="true" http-equiv="X-UA-Compatible" content="IE=edge"/><meta data-react-helmet="true" charset="utf-8"/><meta data-react-helmet="true" name="viewport" content="width=device-width, initial-scale=1"/>\n    \n    \n    <link rel="stylesheet" href="/static/css/bundle.7346228d.css">\n    <link href="/static/css/0.250e8f00.chunk.css" defer rel="stylesheet" type="text/css">\n<link href="/0-feef76b3050030886ad8.css" defer rel="stylesheet" type="text/css">

In [ ]:
cp.callAll(page)

'Salesforce Data AnalystDescriptionBased in the heart of Dublins Silicon Docks Dublin our client has been at the forefront in changing the internet and online technologies as we know itOur client is currently conducting all interviews online and are onboarding all new workers remotelyInitially on a month contract our client requires a SalesforceData Analyst with years of experience in an analytical role with a marketing multinational agency or with a tech multinational The successful candidate will analyze leads from campaigns and events from the marketing team in order to improve the performance of their acquisition campaignJob ReferenceBBBH What will aSalesforceData Analyst doData diagnostics extracting scrubbing and patching very large sets of data together from a variety of internal and external sourcesEffectively communicate your insights and findings to other members of the teamCreate reports and share analysis with the sales teams and stakeholdersWork with all levels of internal

In [ ]:
# To connect google drive
#from google.colab import drive
#drive.mount('/content/drive')